In [106]:
import advent
data: list[str] = advent.get_lines(20)

def parse_rule(rule: str):
    l, r = rule.split(' -> ')
    if l == 'broadcaster':
        ntype, nname = 'b', 'broadcaster'
    else:
        ntype, nname = l[0], l[1:]
    destinations = r.split(', ')
    return nname, ntype, destinations

rules = [parse_rule(rule) for rule in data]
node_dict = dict((a[0], (a[1], a[2])) for a in rules)
#rules, node_dict


In [107]:
def connected_inputs(node_dict: dict[str, tuple[str, list[str]]]):
    # dictionary from nodes to connected input nodes
    result: dict[str, list[str]] = {}
    for key in node_dict:
        for dest in node_dict[key][1]:
            if dest not in result:
                result[dest] = [key]
            else:
                result[dest].append(key)
    return result

In [108]:
from collections import deque
inputs = connected_inputs(node_dict)
# message receiver, plus message type, plus message sender. low = 0, high = 1
messages = deque([('broadcaster', 0, 'input')])
states: dict[str, int|dict[str, int]] = dict((node, 0) for node in node_dict)

for node in node_dict:
    if node_dict[node][0] == '&':
        states[node] = dict((s, 0) for s in inputs[node])

low, high = 0, 0

def handle_message(messages: deque[tuple[str, int, str]], states: dict[str, int|dict[str, int]]):
    # INPLACE updates the message queue by taking a message and processing it
    # also INPLACE updates the states dictionary
    global low
    global high

    message = messages.popleft()
    node, content, sender = message
    if content == 0: low += 1
    elif content == 1: high += 1

    if node not in node_dict:
        return messages, states # this is an output-only node

    node_type, destinations = node_dict[node]

    if node_type == 'b':
        for d in destinations:
            messages.append((d, content, 'broadcaster'))
    elif node_type == '%' and content == 0:
        states[node] = 1 - states[node]
        for d in destinations:
            messages.append((d, states[node], node))
    elif node_type == '&':
        states[node][sender] = content
        m = 0 if all(states[node].values()) else 1
        for d in destinations:
            messages.append((d, m, node))
    return messages, states

In [109]:
for _ in range(1000):
    messages = deque([('broadcaster', 0, 'input')])
    while messages:
        handle_message(messages, states)

low * high

3000000

In [110]:
buttons = 0
for _ in range(100):
    messages = deque([('broadcaster', 0, 'input')])
    buttons += 1
    while messages:
        handle_message(messages, states)
    print(states['o'])

{'c': 0}
{'c': 0}
{'c': 0}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 1}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 0}
{'c': 1}


In [84]:
[z > 0 for z in states['zg'].values()]

[False, False, False, False]